In [2]:
# author: mugalino 2022

# Imports the necessary packages
import numpy as np

# This function implements the Cholesky factorization A = T(R) R
# Recall that T(R) or the transpose of R is just L
# Hence, this is equivalent to A = L T(L)

def cholesky_factor(A, debug = False, check_comparison = False, lower_triangular = False):
    """
    This is a function that takes in an input matrix A
    and returns a matrix L or R such that A = L T(L) or T(R)T. It serves as
    a first step to solving the equation (Ax = b) using Cholesky
    factorization for symmetric positive definite matrices.
    
    mugalino 2022
    """
    
    # INITIAL CHECK IF INPUT IS SPD
    # Check 1 : Symmetric if A = A^T
    # Check 2 : Positive definite if eigenvalues of A are all positive
    
    if type(A) != np.array :
        A = np.array(A)
    
    # Perform checks
    check_one = (A == A.T).all()
    check_two = (np.linalg.eigvals(A) >= np.zeros(len(np.linalg.eigvals(A)))).all()
    
    if check_one:
        if check_two:
            None
        else:
            print("[Validity Check] The input is not positive definite. Cholesky factorization is only for symmetric positive definite matrices.")
            return
    else:
        print("[Validity Check] The input is not symmetric. Cholesky factorization is only for symmetric positive definite matrices.")
        return
    # INITIAL CHECK ENDS HERE
    
    # CHOLESKY FACTORIZATION STARTS HERE
    
    # Since the matrix is symmetric positive definite, we only need the upper/lower triangular matrix
    # However, we don't necessarily simplify the number of operations by just obtaining the 
    # triangular piece. We make sure that the elements we operate is just the necessary
    # parts.
    
    if lower_triangular:
        L = np.zeros(np.shape(A))

        # Note to self: i index for rows, j index for columns

        for i in range(np.shape(A)[0]):

            # We only need pieces to the left of the diagonal for the j "for" loop

            for j in range(i+1):
                # There are two cases for the Cholesky decomposition
                # Case 1: diagonal elements
                # Case 2: non-diagonal elementsx

                # CASE 1
                if i == j:
                    L[i,j] = np.sqrt(A[i,i] - np.sum([L[i,k]**2 for k in range(i)]))

                # CASE 2
                elif i != j:
                    L[i,j] = A[i,j] - np.sum([L[j,k] * L[i,k] for k in range(j)])
                    L[i,j] = L[i,j] / L[j,j]
        cholesky = L
        
    else:
        R = np.zeros(np.shape(A))
        
        for i in range(len(R)):
            R[i,i:] = A[i,i:]
        
        for i in range(np.shape(A)[0]):
            
            # We only need pieces to the right of the diagonal for the j "for" loop
            for j in range(i+1,np.shape(A)[0]):
                m = R[i,j] / R[i,i]
                R[j, j:] = R[j,j:] - m * R[i,j:]
                
            R[i,i:] = R[i,i:] / np.sqrt(R[i,i])
            
        cholesky = R
        
    # CHOLESKY FACTORIZATION ENDS HERE
    
    # FINAL CHOLESKY CHECK
    # Let us use a numpy package for linear algebra to check our result
    
    if check_comparison:
        print("Error matrix from input (A - L * T(L)): \n", A - np.dot(L, L.T))
        print("Error matrix from numpy LAPACK: \n", L - np.linalg.cholesky(A))
    
    # Checks the error if lower than tolerance ~machine precision
    if (np.abs(cholesky - np.linalg.cholesky(A)) < 1.e-15).all:
        return cholesky
    else:
        print(r"Error is higher than the tolerance value $\varepsilon = 10^{-15}$")  

def forward_substitution(L, b):
    """
    This is a function that takes in a lower triangular matrix 
    L and input vector b to obtain the solution of the matrix equation
    (Ly = b) using forward substitution.
    """
    # START OF FORWARD SUBSTITUTION STEP
    y_vector = np.zeros(len(b))
    
    for index in range(len(y_vector)):
        y_vector[index] = (b[index] - np.sum(y_vector[:index]*L[index,:index]))/L[index, index]
    
    # END OF FORWARD SUBSTITUTION STEP
    
    return y_vector

def backward_substitution(R, y):
    """
    This is a function that takes in an upper triangular matrix 
    R and input vector y to obtain the solution of the matrix equation
    (Rx = y) using backward substitution.
    """
    # START OF BACKWARD SUBSTITUTION STEP
    solution_vector = np.zeros(len(y))
    
    for index in range(len(solution_vector)-1,-1,-1):
        solution_vector[index] = (y[index] - np.sum(solution_vector[index:]*R[index,index:]))/R[index, index]
    
    # END OF BACKWARD SUBSTITUTION STEP
    
    return solution_vector

def cholesky_solver(A, b, debug = False, check_comparison = False, lower_triangular = True):
    """
    This is a function that takes in an input matrix A
    and returns the solution vector x to the matrix equation (Ax = b) 
    using Cholesky factorization for symmetric positive definite matrices.
    
    mugalino 2022
    """
    
    # Obtain lower triangular matrix
    lower = cholesky_factor(A, debug, check_comparison, lower_triangular)
    # and upper triangular matrix
    upper = lower.T
    
    y_vector = forward_substitution(lower, b)
    solution_vector = backward_substitution(upper, y_vector)

    if check_comparison:
        print("Solution using NUMPY LAPACK:", np.linalg.solve(A,b))
        print("Solution using our Cholesky Solver:", solution_vector)
        print("Error array:", np.abs(np.linalg.solve(A,b) - solution_vector))
    
    print("RMS Error:", np.sqrt(np.average((np.linalg.solve(A,b) - solution_vector)**2)))
    print("Solution using our Cholesky Solver:", solution_vector)

In [3]:
cholesky_solver([[5,2],[2,3]], [1,2], debug = False, check_comparison = False, lower_triangular = True)

RMS Error: 0.0
Solution using our Cholesky Solver: [-0.09090909  0.72727273]


In [4]:
input_matrix_A = [[4,1,1,1],[1,3,-1,1],[1,-1,2,0],[1,1,0,2]]
vector_b = [0.65, 0.05, 0.00, 0.50]
cholesky_solver(input_matrix_A, vector_b, debug = False, check_comparison = False, lower_triangular = True)
np.linalg.solve(input_matrix_A, vector_b)

RMS Error: 3.1031676915590914e-17
Solution using our Cholesky Solver: [ 0.2  -0.2  -0.2   0.25]


array([ 0.2 , -0.2 , -0.2 ,  0.25])